In [0]:
# =========================
# (1) LIMPIEZA DE WIDGETS
# =========================
# Los widgets son parámetros editables en el notebook (referenciar a un ADLS, o el cambio de un storage account sin tocar código) 
# Permiten que el entregable sea portable y limpio, ya que los parámetros son asignados por usuarios (como Stored Procedure en SSMS),
# Y son geniales para la automatización del workflow
dbutils.widgets.removeAll()  # Elimina widgets anteriores para evitar conflictos


In [0]:
# =========================
# (2) PARÁMETROS DEL AMBIENTE
# =========================
dbutils.widgets.text("storage_account", "adlssmartdata0303ev", "ADLS Storage Account")  # Nombre del Storage Account - Azure Data Lake
dbutils.widgets.text("container", "raw", "ADLS Raw Container")  # Container dentro del Storage Account. De acá se extraerán los datos crudos hacia LOAD
#dbutils.widgets.text("base_path", "smartdata", "Base path")  # Carpeta raíz dentro del container (ej: smartdata)
dbutils.widgets.text("catalog_name", "catalog_footballdata", "Unity Catalog name")  # Nombre del catálogo en Unity Catalog
dbutils.widgets.text("storage_credential", "credential", "UC Storage Credential")  # Nombre del Storage Credential (debe existir en tu workspace)

# =========================
# (3) LECTURA DE WIDGETS
# =========================
storage_account = dbutils.widgets.get("storage_account")  # Captura el nombre real del Storage Account en una variable (aplica a todos)
container = dbutils.widgets.get("container")              # Captura el container real
# base_path = dbutils.widgets.get("base_path").strip("/")   # Normaliza la ruta (sin / inicial o final)
catalog_name = dbutils.widgets.get("catalog_name")        # Capturamos el nombre del catálogo
storage_credential = dbutils.widgets.get("storage_credential")  # Captura el Storage Credential (Unity Catalog)

# =========================
# (4) CONSTRUCCIÓN DE URL ABFSS
# =========================
adls_base_url = f"abfss://{container}@{storage_account}.dfs.core.windows.net/" # {base_path}"  # Ruta base en ADLS Gen2 usando ABFSS
print("ADLS Base URL:", adls_base_url)  # Log para validar que apunta a TU ADLS y ruta correcta
# Esto crea/actualiza un widget invisible que SQL sí puede leer con la sintaxis ${...}
dbutils.widgets.text("adls_base_url", adls_base_url)

In [0]:
%sql
-- =========================
-- (1) BORRAR TODO EL CATÁLOGO (REINICIO LIMPIO)
-- =========================
-- Esto elimina schemas/objetos dentro del catálogo; considerar eliminarlo luego del primer run en ambiente producción?
-- Descripción: Contenedor padre de más alto nivel para organizar datos/objetos (schemas, tablas, views, volumens) ~ referenciar las notas de la clase UnitCatalog
DROP CATALOG IF EXISTS ${catalog_name} CASCADE;

-- =========================
-- (2) CREAR CATÁLOGO (CONTENEDOR TOP-LEVEL EN UNITY CATALOG)
-- =========================
CREATE CATALOG IF NOT EXISTS ${catalog_name};

-- =========================
-- (3) CREAR SCHEMAS POR CAPA (MEDALLION)
-- =========================
-- Subdisivión dentro del catalogo (carpetas lógicas para tablas/volumes). Se separan por Medallion/Lakehouse
CREATE SCHEMA IF NOT EXISTS ${catalog_name}.raw;          -- Datos fuente (sin transformar o mínimamente organizados)
CREATE SCHEMA IF NOT EXISTS ${catalog_name}.bronze;       -- Ingesta raw “tal cual” pero ya en Delta/tabla, con auditoría básica
CREATE SCHEMA IF NOT EXISTS ${catalog_name}.silver;       -- Transformaciones aplicas; Limpieza/estandarización + reglas de negocio + deduplicación
CREATE SCHEMA IF NOT EXISTS ${catalog_name}.golden;       -- Agregaciones: Tablas finales para BI/consumo (métricas, agregaciones, data marts)
CREATE SCHEMA IF NOT EXISTS ${catalog_name}.exploratory;  -- Sandboxes para pruebas / EDA sin afectar producción

-- =========================
-- (4) CREAR UNA EXTERNAL LOCATION (ENLAZA UC -> ADLS) - Revisar Video Clase
-- =========================
-- Forma oficial de que Unity Catalog apunte a ADLS de manera gobernada; permite que el acceso al storage esté controlado por Unity Catalog
-- (permisos, auditoría, seguridad)
-- Requiere que el Storage Credential ya exista y tenga acceso al container/ruta.
-- !! Ya que ya tenemos un external location conectado a la misma carpeta en ADLS. Reciclar esa para no introducir complicaciones
-- CREATE EXTERNAL LOCATION IF NOT EXISTS extloc_smartdata_raw
-- URL '${adls_base_url}raw'
-- WITH (STORAGE CREDENTIAL `${storage_credential}`);

-- =========================
-- (5) CREAR VOLUME (ESPACIO TIPO “FOLDER” ADMINISTRADO POR UC)
-- =========================
-- Este volume vive en el schema RAW y físicamente apunta al storage en ADLS (vía LOCATION).
CREATE EXTERNAL VOLUME IF NOT EXISTS ${catalog_name}.raw.datasets
LOCATION '${adls_base_url}raw/datasets';

---- =========================
---- (6) VALIDACIONES RÁPIDAS
---- =========================
DESCRIBE VOLUME ${catalog_name}.raw.datasets;  -- Muestra metadata y ubicación real del volume
SHOW VOLUMES IN ${catalog_name}.raw;          -- Lista volumes disponibles en el schema raw
SHOW SCHEMAS IN ${catalog_name};              -- Lista schemas creados en el catálogo


In [0]:
%sql
DESCRIBE EXTERNAL LOCATION `exlt-raw`;